<a href="https://colab.research.google.com/github/firmai/random-assets/blob/master/Linear_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap

     |████████████████████████████████| 266kB 3.5MB/s 
  Created wheel for shap: filename=shap-0.33.0-cp36-cp36m-linux_x86_64.whl size=382248 sha256=5741555ca4e318f13cda7a9685cd73fad1ad606548c2345d5598a3e1344551fb
  Stored in directory: /root/.cache/pip/wheels/39/0f/88/a8124d43431284e10f263ffe449e119344c6145c3a165d186c
Successfully built shap


In [2]:
import pandas as pd 
import shap
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import numpy as np

X_pr = pd.read_csv("https://github.com/firmai/random-assets/blob/master/data_clean.csv?raw=true").set_index("Unnamed: 0", drop=True)

Using TensorFlow backend.


In [0]:
X_pr.to_csv("frame.csv")

In [0]:
#X_pr = pd.read_csv()

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.optimizers import Adam




In [10]:
int(first.shape[1]/11)

4

In [8]:
4*8

32

In [0]:
X_pr.index.name = 'index'
target = "price"
first = X_pr.sample(int(len(X_pr)/2))
second = X_pr[~X_pr.isin(first)].dropna()


def modelling(first, second, target, inter_portion=0.5, sqr_portion=0.5, contribution_portion=0.9, final_contribution=0.9, node_size=1.0, final=False, modeller="LightGBM"):
  
  cols_drop = [target ]

  def shap_frame(first,second, target, node_size):

    #could also use linear booster XGBoost
    d_train = lgb.Dataset(first.drop(columns=[target]), label=first[target])
    d_valid = lgb.Dataset(second.drop(columns=[target]), label=second[target])
    params = {
      
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmsle',
        'max_depth': 6, 
        'learning_rate': 0.1,
        'verbose': 0,
      'num_threads':16}
    n_estimators = 100

    model = lgb.train(params, d_train, 100, verbose_eval=1)

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(first.drop([target], axis=1))
    shap_fram = pd.DataFrame(shap_values[:,:], columns=list(first.drop([target], axis=1).columns))
    shap_new = shap_fram.sum().sort_values().to_frame()
    print("Finished TreeExplainer")
    
    return shap_new, explainer, model

  def shap_frame_keras(first,second, target, node_size):
    ## deepexplainer and gradientexplainer has no interaction_values

    # Set the input shape
    input_shape = (len(first.columns)-1,)
    print(f'Feature shape: {input_shape}')

    # Create the model
    model = Sequential()
    model.add(Dense(int(int(first.shape[1]/11)*node_size/(contribution_portion/8)), input_shape=input_shape, activation='relu'))
    model.add(Dense(int((int(first.shape[1]/11)*node_size/2)/(contribution_portion/8)), activation='relu'))
    model.add(Dense(1, activation='linear'))


    # Configure the model and start training
    # validation_data=(X_test, Y_test)


    model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.01), metrics=['mean_squared_error'])

    stoppy = EarlyStopping(monitor='val_loss', patience=20)

    model.fit(first.drop(columns=[target]), first[target], epochs=1000, batch_size=10, verbose=1, validation_split=0.2, callbacks=[stoppy])

    #explainer = shap.DeepExplainer(model,first.drop(columns=[target]).values)

    explainer  = shap.GradientExplainer(model,first.drop(columns=[target]).values)

    shap_values = explainer.shap_values(first.drop([target], axis=1).values)
    shap_fram = pd.DataFrame(shap_values[0][:,:], columns=list(first.drop([target], axis=1).columns))
    shap_new = shap_fram.sum().sort_values().to_frame()

    return shap_new, explainer, model

  if modeller=="Keras":
    shape_frame_all = shap_frame_keras
  else:
    shape_frame_all = shap_frame

  shap_new, explainer, model = shape_frame_all(first,second,target,node_size)

  shap_new_abs = shap_new[0].abs()
  shap_new_abs = shap_new_abs.sort_values(ascending=False)
  main_ft = shap_new_abs[shap_new_abs.cumsum().sub((shap_new_abs.sum()*sqr_portion)).le(0)]

  preds = model.predict(second.drop(columns=[target]))
  mse = mean_squared_error(second[target], preds)
  print(mse)

  def main_calc(new_df,main_ft):
    df_square = new_df[list(main_ft.index)]
    sqr_name = [fa+"_POWER_2" for fa in df_square.columns]
    log_p_name = [fa+"_LOG_p_one_abs" for fa in df_square.columns]
    rec_p_name = [fa+"_RECIP_p_one" for fa in df_square.columns]
    sqrt_name = [fa+"_SQRT_p_one" for fa in df_square.columns]

    df_sqr = pd.DataFrame(np.power(df_square.values, 2),columns=sqr_name, index=new_df.index)
    df_log = pd.DataFrame(np.log(df_square.add(1).abs().values),columns=log_p_name, index=new_df.index)
    df_rec = pd.DataFrame(np.reciprocal(df_square.add(1).values),columns=rec_p_name, index=new_df.index)
    df_sqrt = pd.DataFrame(np.sqrt(df_square.abs().add(1).values),columns=sqrt_name, index=new_df.index)

    dfs = [df_sqr, df_log, df_rec, df_sqrt]

    df_connect=  pd.concat(dfs, axis=1)

    return df_connect

  ## An attempt to decrease the amount of interactin features.

  select_ft = shap_new_abs[shap_new_abs.cumsum().sub((shap_new_abs.sum()*contribution_portion)).le(0)]
  select_ft = list(select_ft.index)
  select_ft.append(target)

  ## has to remain tree for interaction effects. shap_frame
  shap_select, explainer, model = shap_frame(first[select_ft], second[select_ft], target, node_size )
  shap_select_abs = shap_select[0].abs()

  ### Interactions Features
  shap_interaction_values = explainer.shap_interaction_values(first[select_ft].drop(cols_drop, axis=1))

  shap_interaction_values_abs = abs(shap_interaction_values)
  df_start = pd.DataFrame(np.sum(shap_interaction_values_abs ,axis=0),columns=first[select_ft].drop(cols_drop, axis=1).columns, index=first[select_ft].drop(cols_drop, axis=1).columns)

  #the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
  sol = (df_start.where(np.triu(np.ones(df_start.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))
  #first element of sol series is the pair with the bigest correlation

  ## New Data Frames From Feature Interaction and Main Feature
  ## If you have features [a, b, c] the default polynomial features(in sklearn the degree is 2) should be [1, a, b, c, a^2, b^2, c^2, ab, bc, ca].

  ## Interaction Calculations

  dab = sol[sol.cumsum().sub((sol.sum()*inter_portion)).le(0)]

  list_one = [da[0] for da in dab.index]
  list_two = [da[1] for da in dab.index]

  def inter_cal(list_one, list_two,new_df):

    mult = [ra+"_X_"+ba for ra, ba in zip(list_one, list_two)]
    div = [ra+"_DIV_"+ba for ra, ba in zip(list_one, list_two)]
    print("len one " + str(len(list_one)) )
    print("len two " + str(len(list_two)) )
    inter_mult = pd.DataFrame(new_df[list_one].values*new_df[list_two].values, columns=mult, index=new_df.index)
    div_p_one = pd.DataFrame(new_df[list_one].add(1).values/new_df[list_two].add(1).values, columns=div, index=new_df.index)

    df_one = pd.concat((inter_mult,div_p_one), axis=1)

    return df_one


  def combine(target, list_one, list_two, main_ft, new_df):

    inter_mult = inter_cal(list_one, list_two, new_df.drop(columns=[target]))

    df_sqr = main_calc(new_df.drop(columns=[target]),main_ft)

    new = pd.concat((inter_mult,df_sqr),axis=1)
    new2 = pd.concat((new_df,new),axis=1)

    new2 = new2.loc[:,~new2.columns.duplicated()]

    return new2

  new_first = combine(target, list_one, list_two, main_ft, first)

  new_second = combine(target, list_one, list_two, main_ft, second)

  if final:
    print("final")
    shap_select, explainer, model = shape_frame_all(new_first, new_second,target,node_size )
    ### Can be put into function, somewhat unnecessary
    shap_select_abs = shap_select[0].abs()
    shap_select_abs = shap_select_abs.sort_values(ascending=False)
    final_ft = shap_select_abs[shap_select_abs.cumsum().sub((shap_select_abs.sum()*final_contribution)).le(0)]
    final_ft = list(final_ft.index)
    final_ft.append(target)
    return new_first,new_second, mse,final_ft, shap_select_abs

  return new_first,new_second, mse

In [0]:
import gc
## Definitely a benefit to run twice but not three times. 
def runner(first_run, second_run, target, inter_portion=0.8, sqr_portion=0.9, contribution_portion=0.9, final_contribution=0.95, deflator=0.7, node_size=1.0 , runs=2, modeller="LightGBM"):
#def runner(first_run, second_run, target, inter_portion=0.4, sqr_portion=0.3, contribution_portion=0.9, deflator=0.6, runs=3):
  for r in range(runs):
    r += 1
    if r ==runs:
      print("final")
      first_run, second_run, mse, shapper, shap_select_abs = modelling(first_run, second_run, target, inter_portion, sqr_portion, contribution_portion, final_contribution,node_size, True, modeller)
    else:
      first_run, second_run, mse = modelling(first_run, second_run, target, inter_portion, sqr_portion, contribution_portion, final_contribution, node_size, False, modeller)

    inter_portion = inter_portion * deflator
    sqr_portion = sqr_portion * deflator * 1.1
    contribution_portion = contribution_portion * deflator
    gc.collect()
  
  return first_run, second_run, shapper, shap_select_abs

In [204]:
new_first_deep, new_second_deep, shapper_deep, shap_select_abs_deep = runner(first, second, target, modeller="Keras")

Feature shape: (53,)
Train on 4180 samples, validate on 1046 samples
Epoch 1/100
4180/4180 [==============================] - 1s 203us/step - loss: 86.7481 - mean_squared_error: 25311.0536 - val_loss: 77.3674 - val_mean_squared_error: 19551.2793
Epoch 2/100
4180/4180 [==============================] - 1s 136us/step - loss: 76.7712 - mean_squared_error: 21917.7316 - val_loss: 68.8881 - val_mean_squared_error: 17909.7802
Epoch 3/100
4180/4180 [==============================] - 1s 131us/step - loss: 71.6620 - mean_squared_error: 19328.0037 - val_loss: 67.1379 - val_mean_squared_error: 17561.8642
Epoch 4/100
4180/4180 [==============================] - 1s 133us/step - loss: 71.4044 - mean_squared_error: 19126.8401 - val_loss: 64.8086 - val_mean_squared_error: 15749.8226
Epoch 5/100
4180/4180 [==============================] - 1s 141us/step - loss: 69.4694 - mean_squared_error: 18455.2205 - val_loss: 67.0606 - val_mean_squared_error: 15105.6256
Epoch 6/100
4180/4180 [=======================

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Finished TreeExplainer
len one 41
len two 41
len one 41
len two 41
final
Feature shape: (219,)
Train on 4180 samples, validate on 1046 samples
Epoch 1/100
4180/4180 [==============================] - 1s 325us/step - loss: 1250.4924 - mean_squared_error: 293927418.1353 - val_loss: 162.5181 - val_mean_squared_error: 61902.2344
Epoch 2/100
4180/4180 [==============================] - 1s 233us/step - loss: 150.2068 - mean_squared_error: 56800.3353 - val_loss: 139.7689 - val_mean_squared_error: 50363.1580
Epoch 3/100
4180/4180 [==============================] - 1s 234us/step - loss: 146.1095 - mean_squared_error: 59529.9556 - val_loss: 137.4926 - val_mean_squared_error: 48940.0446
Epoch 4/100
4180/4180 [==============================] - 1s 224us/step - loss: 142.3763 - mean_squared_error: 53100.3841 - val_loss: 138.8018 - val_mean_squared_error: 49991.9513
Epoch 5/100
4180/4180 [==============================] - 1s 236us/step - loss: 141.0518 - mean_squared_error: 58894.5004 - val_loss: 133

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Finished TreeExplainer
len one 1
len two 1
len one 1
len two 1
final
Feature shape: (237,)
Train on 4180 samples, validate on 1046 samples
Epoch 1/100
4180/4180 [==============================] - 2s 367us/step - loss: 173275345974.8756 - mean_squared_error: 12857250968910950352027648.0000 - val_loss: 119510946826.7636 - val_mean_squared_error: 2710593754457381250531328.0000
Epoch 2/100
4180/4180 [==============================] - 1s 252us/step - loss: 41360044201.8635 - mean_squared_error: 4202390978520230884540416.0000 - val_loss: 159.7057 - val_mean_squared_error: 58685.1349
Epoch 3/100
4180/4180 [==============================] - 1s 257us/step - loss: 398.5300 - mean_squared_error: 236092895.9618 - val_loss: 156.4429 - val_mean_squared_error: 57386.8538
Epoch 4/100
4180/4180 [==============================] - 1s 259us/step - loss: 292080.1952 - mean_squared_error: 353322957003485.6250 - val_loss: 152.4581 - val_mean_squared_error: 55703.4737
Epoch 5/100
4180/4180 [==================

In [279]:
new_first_tree, new_second_tree, shapper_tree, shap_select_abs_tree = runner(first, second, target, modeller="LightGBM")

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Finished TreeExplainer
11639.838583703673


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Finished TreeExplainer
len one 22
len two 22
len one 22
len two 22
final
Finished TreeExplainer
11997.976374663187


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Finished TreeExplainer
len one 20
len two 20
len one 20
len two 20
final
Finished TreeExplainer


In [0]:
import pandas as pd
from sklearn import preprocessing

def all(new_first_two,new_second_two, shapper,target, dall=False ):
  def scaler(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, index=df.index, columns=df.columns)
    return df

  if dall:
    shapper = new_first_two.columns
  

  new_first_y = new_first_two[target].copy()
  new_first = new_first_two[shapper].copy()
  new_first = scaler(new_first.drop([target],axis=1)).copy()

  new_second_y = new_second_two[target].copy()
  new_second = new_second_two[shapper].copy()
  new_second = scaler(new_second.drop([target],axis=1)).copy() 

  return new_first,new_first_y, new_second, new_second_y

In [0]:
new_deep_one,new_first_y, new_deep_two, new_second_y = all(new_first_deep,new_second_deep, shapper_deep,target , True)

In [0]:
new_first = new_deep_one.copy()
new_second = new_deep_two.copy()

In [0]:
# tree
new_tree_one,_, new_tree_two,_  = all(new_first_tree,new_second_tree, shapper_tree,target, True)
new_first = new_tree_one.copy()
new_second = new_tree_two.copy()

In [281]:
lass = LassoLarsCV(cv=5).fit(new_first, new_first_y)

Regressors in active set degenerate. Dropping a regressor, after 339 iterations, i.e. alpha=1.105e-04, with an active set of 143 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 340 iterations, alpha=1.091e-04, previous alpha=1.067e-04, with an active set of 143 regressors.
Regressors in active set degenerate. Dropping a regressor, after 469 iterations, i.e. alpha=1.663e-05, with an active set of 171 regressors, and the smallest cholesky pivot element being 4.215e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 499 iterations, i.e. alpha=1.173e-05, with an active set of 171 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, a

In [282]:
lass

LassoLarsCV(copy_X=True, cv=5, eps=2.220446049250313e-16, fit_intercept=True,
            max_iter=500, max_n_alphas=1000, n_jobs=None, normalize=True,
            positive=False, precompute='auto', verbose=False)

In [283]:
## Tree
preds = lass.predict(new_second)
mse = mean_squared_error(new_second_y, preds)
print(mse)

16241.6754428426


In [271]:
preds = lass.predict(new_second)
mse = mean_squared_error(new_second_y, preds)
print(mse)

15129.27870045936


In [0]:
# mse = pd.DataFrame(columns=["alpha","mse"])
# mse["alpha"]= [ra.sum() for ra in lass.alphas_]
# mse["mse"] = [ra.sum() for ra in lass.mse_path_]
# mse.sort_values("mse").to_csv("dd.csv")

In [289]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1, tol =0.1)
lass = clf.fit(new_first,new_first_y)
preds = lass.predict(new_second)
mse = mean_squared_error(new_second_y, preds)
print(mse)

# #12381 0.10


13448.410623256994


In [0]:
from sklearn.feature_selection import SelectFromModel
# max_features

#new_first = new_deep_one.copy()
#new_second = new_deep_two.copy()

model = SelectFromModel(lass, prefit=True)
X_train = new_first[new_first.columns[model.get_support(indices=True)].copy()].copy()
#new_first_y = new_first_two[target]
X_test = new_second[list(X_train.columns)].copy()
#new_second_y = new_second_two[target]


In [292]:
X_train.loc[:,~X_train.columns.duplicated()].shape

(5226, 84)

In [286]:
X_test.columns

Index(['review_scores_rating', 'number_of_reviews', 'minimum_nights',
       'security_deposit', 'beds', 'availability_365', 'bedrooms_per_person',
       'Bondi', 'Coogee', 'North Bondi', 'Randwick', 'Redfern', 'Sydney',
       'Boutique hotel', 'Bungalow', 'Condominium', 'Cottage', 'Guest suite',
       'Guesthouse', 'Hostel', 'Hotel', 'House', 'Other', 'Serviced apartment',
       'Townhouse', 'Villa', 'Hotel room', 'Shared room', 'luxury_moderate',
       'moderate', 'strict_14_with_grace_period', 'super_strict_30',
       'super_strict_60', 'security_deposit_X_cleaning_fee',
       'bedrooms_X_cleaning_fee', 'bedrooms_X_beds',
       'bathrooms_X_security_deposit',
       'cleaning_fee_X_past_and_future_popularity',
       'Entire home/apt_X_bathrooms', 'Entire home/apt_X_cleaning_fee',
       'accommodates_X_Private room', 'bathrooms_DIV_bedrooms',
       'security_deposit_DIV_cleaning_fee', 'bedrooms_DIV_beds',
       'bathrooms_DIV_security_deposit', 'Entire home/apt_DIV_bathro

In [274]:
## Deep

from sklearn import linear_model
lm = linear_model.LinearRegression()
lm = lm.fit(X_train,new_first_y)
preds = lm.predict(X_test)

mse = mean_squared_error(new_second_y, preds)
print(mse)

#12381 0.10

13756.481939623793


In [293]:
## Tree

from sklearn import linear_model
lm = linear_model.LinearRegression()
lm = lm.fit(X_train,new_first_y)
preds = lm.predict(X_test)

mse = mean_squared_error(new_second_y, preds)
print(mse)

#12381 0.10

15003.467159303042


In [190]:
## Tree

from sklearn import linear_model
lm = linear_model.LinearRegression()
lm = lm.fit(X_train,new_first_y)
preds = lm.predict(X_test)

mse = mean_squared_error(new_second_y, preds)
print(mse)

#12381 0.10

13163.966378637819


In [157]:
preds

array([ 19.24207033, 381.4889417 , 315.06818663, ..., 204.10551279,
       118.35484246, 106.50990425])

In [0]:
import pandas as pd
from sklearn import preprocessing

def scaler(df):
  x = df.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  df = pd.DataFrame(x_scaled)
  return df

add_first_y = first[target]
add_first = scaler(first.drop([target],axis=1))

add_second_y = second[target]
add_second = scaler(second.drop([target],axis=1)) 


In [288]:
from sklearn import linear_model
#clf = linear_model.Lasso(alpha=0.4)
clf = linear_model.LinearRegression()
preds = clf.fit(add_first,add_first_y).predict(add_second)
mse = mean_squared_error(add_second_y, preds)
print(mse)


14287.356235987374


In [78]:
from sklearn import linear_model
#clf = linear_model.Lasso(alpha=0.4)
clf = linear_model.LinearRegression()
preds = clf.fit(add_first,add_first_y).predict(add_second)
mse = mean_squared_error(add_second_y, preds)
print(mse)


KeyError: ignored

In [0]:
# from sklearn.preprocessing import PolynomialFeatures
# poly_reg = PolynomialFeatures(degree=3)
# PR = poly_reg.fit(add_first)

# X_poly = PR.transform(add_first)
# X_pred = PR.transform(add_second)

# pol_reg = linear_model.LinearRegression()
# pol_reg.fit(X_poly, add_first_y)
# preds = pol_reg.predict(X_pred)
# mse = mean_squared_error(add_second_y, preds)
# print(mse)

In [0]:


# shapper_tree = shapper_tree[:10]
# shapper_tree.append(target)

new_tree_one,_, new_tree_two,_  = all(new_first_tree,new_second_tree, shapper_tree,target, True)
new_first = new_tree_one.copy()
new_second = new_tree_two.copy()


# ###  comes here


# new_first =  pd.concat((new_tree_one,new_deep_one ),axis=1)
# new_second =  pd.concat((new_tree_two,new_deep_two ),axis=1)

# new_first = new_first.loc[:,~new_first.columns.duplicated()].copy()
# new_second = new_second.loc[:,~new_second.columns.duplicated()].copy()

# ### comes here

# new_first = new_tree_one.copy()
# new_second = new_tree_two.copy()